Système de Classification Automatique de Livres
On utilise la classification zéro-shot pour catégoriser les descriptions

In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
from tqdm import tqdm

C:\Users\barry\PycharmProjects\projet\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Charger le fichier CSV
df = pd.read_csv("livres_nettoyes.csv")

print("Données chargées avec succès")
print(f"Nombre total de livres: {len(df)}")
print(f"Colonnes: {list(df.columns)}")

Données chargées avec succès
Nombre total de livres: 5197
Colonnes: ['isbn13', 'isbn10', 'title', 'authors', 'categories', 'thumbnail', 'description', 'published_year', 'average_rating', 'num_pages', 'ratings_count', 'titre_complet', 'description_avec_id']


In [3]:
# Catégories et leur fréquence
dist_categories = df["categories"].value_counts()
print("\n10 catégories les plus fréquentes:")
print(dist_categories.head(10))


10 catégories les plus fréquentes:
categories
Fiction                      2111
Juvenile Fiction              390
Biography & Autobiography     311
History                       207
Literary Criticism            124
Religion                      117
Philosophy                    117
Comics & Graphic Novels       116
Drama                          86
Juvenile Nonfiction            57
Name: count, dtype: int64


In [4]:
# Combien de catégories ont peu de livres, l'inverse aussi peut etre fait ici
categories_rares = (dist_categories < 50).sum()
print(f"\nCatégories avec moins de 50 livres: {categories_rares}")


Catégories avec moins de 50 livres: 466


In [5]:
# Vérifier les valeurs manquantes
print("Valeurs manquantes par colonne:")
print(df.isnull().sum())

Valeurs manquantes par colonne:
isbn13                   0
isbn10                   0
title                    0
authors                 32
categories              30
thumbnail              166
description              0
published_year           0
average_rating           0
num_pages                0
ratings_count            0
titre_complet            0
description_avec_id      0
dtype: int64


In [7]:
# Définir comment grouper les catégories
schema_categories = {
    'Fiction': "Fiction",
    'Juvenile Fiction': "Fiction enfant",
    'Comics & Graphic Novels': "Fiction",
    'Drama': "Fiction",
    'Poetry': "Fiction",
    'Biography & Autobiography': "Nonfiction",
    'History': "Nonfiction",
    'Literary Criticism': "Nonfiction",
    'Philosophy': "Nonfiction",
    'Religion': "Nonfiction",
    'Science': "Nonfiction",
    'Juvenile Nonfiction': "Nonfiction enfant"
}

print("Schéma de simplification des catégories:")
for cat_original, cat_simple in schema_categories.items():
    print(f"  {cat_original:30} -> {cat_simple}")

Schéma de simplification des catégories :
  Fiction                        -> Fiction
  Juvenile Fiction               -> Fiction enfant
  Comics & Graphic Novels        -> Fiction
  Drama                          -> Fiction
  Poetry                         -> Fiction
  Biography & Autobiography      -> Nonfiction
  History                        -> Nonfiction
  Literary Criticism             -> Nonfiction
  Philosophy                     -> Nonfiction
  Religion                       -> Nonfiction
  Science                        -> Nonfiction
  Juvenile Nonfiction            -> Nonfiction enfant


In [8]:
# Créer une nouvelle colonne avec les catégories simplifiées
df["cat_simple"] = df["categories"].map(schema_categories)

# Voir les résultats
print("\nNombre de livres par catégorie simplifiée:")
print(df["cat_simple"].value_counts())


Nombre de livres par catégorie simplifiée :
cat_simple
Fiction              2364
Nonfiction            932
Fiction enfant        390
Nonfiction enfant      57
Name: count, dtype: int64


In [9]:
!pip install torch

In [10]:
import torch
from transformers import pipeline
import pandas as pd
from tqdm import tqdm

In [14]:
# Pour la classification
classificateur = pipeline(
    task="zero-shot-classification",
    model="facebook/bart-large-mnli"
)

Device set to use cpu


In [15]:
# Exemple
# Choisir une description de test
test_idx = 10
description_test = df.loc[test_idx, "description"]

# Afficher la description
print(f"Description de test:\n{description_test[:150]}...\n")

# Faire une prédiction
categories_possibles = ["Fiction", "Nonfiction"]
resultat = classificateur(description_test, categories_possibles)

print("Résultat de la classification:")
print(f"  Catégorie prédite: {resultat['labels'][0]}")
print(f"  Confiance: {resultat['scores'][0]:.2%}")

Description de test :
Available in the U.S. for the first time, this is the second volume in the exceptional Legends of the Riftwar series from "New York Times"-bestselling...

Résultat de la classification :
  Catégorie prédite : Fiction
  Confiance : 86.70%


In [16]:
# Fonction pour classifier une description dans une des catégories
# Parametres:
#description: le texte à classifier
#categories_list: liste des catégories possibles
#et on retourne un tuple (catégorie_prédite, score_confiance)
def classifier_description(description, categories_list):

    try:
        if pd.isna(description) or description == "":
            return None, 0.0

        prediction = classificateur(str(description), categories_list)
        categorie = prediction["labels"][0]
        confiance = prediction["scores"][0]

        return categorie, confiance
    except Exception as e:
        print(f"Erreur: {e}")
        return None, 0.0

In [17]:
print("Évaluation sur 300 livres de Fiction")

predictions_fiction = []
vraies_etiquettes = []
scores_confiance_fiction = []

# Récupérer les descriptions de fiction
fiction_df = df[df["cat_simple"] == "Fiction"].reset_index(drop=True)

# Prédire pour 300 livres
for i in tqdm(range(min(300, len(fiction_df)))):
    desc = fiction_df.loc[i, "description"]
    cat, score = classifier_description(desc, ["Fiction", "Nonfiction"])

    predictions_fiction.append(cat)
    vraies_etiquettes.append("Fiction")
    scores_confiance_fiction.append(score)

Évaluation sur 300 livres de Fiction...


100%|██████████| 300/300 [08:08<00:00,  1.63s/it]


In [18]:
print("\nÉvaluation sur 300 livres de Nonfiction")

predictions_nonfiction = []
vraies_etiquettes_non = []
scores_confiance_non = []

# Récupérer les descriptions de nonfiction
nonfiction_df = df[df["cat_simple"] == "Nonfiction"].reset_index(drop=True)

# Prédire pour 300 livres
for i in tqdm(range(min(300, len(nonfiction_df)))):
    desc = nonfiction_df.loc[i, "description"]
    cat, score = classifier_description(desc, ["Fiction", "Nonfiction"])

    predictions_nonfiction.append(cat)
    vraies_etiquettes_non.append("Nonfiction")
    scores_confiance_non.append(score)


Évaluation sur 300 livres de Nonfiction...


100%|██████████| 300/300 [07:30<00:00,  1.50s/it]


In [19]:
# Combiner fiction et nonfiction
toutes_predictions = predictions_fiction + predictions_nonfiction
toutes_vraies = vraies_etiquettes + vraies_etiquettes_non
tous_scores = scores_confiance_fiction + scores_confiance_non

# Créer un DataFrame avec les résultats
df_eval = pd.DataFrame({
    "vraie_categorie": toutes_vraies,
    "categorie_predite": toutes_predictions,
    "score": tous_scores
})

print("\nAperçu des résultats:")
print(df_eval.head(10))


Aperçu des résultats :
  vraie_categorie categorie_predite     score
0         Fiction           Fiction  0.843827
1         Fiction           Fiction  0.505523
2         Fiction           Fiction  0.535262
3         Fiction        Nonfiction  0.606821
4         Fiction           Fiction  0.654583
5         Fiction           Fiction  0.507003
6         Fiction           Fiction  0.829240
7         Fiction           Fiction  0.984461
8         Fiction           Fiction  0.977928
9         Fiction           Fiction  0.508185


In [20]:
# Vérifier les prédictions correctes
df_eval["correct"] = (df_eval["vraie_categorie"] == df_eval["categorie_predite"]).astype(int)

# Calculs
total = len(df_eval)
corrects = df_eval["correct"].sum()
accuracy = corrects / total

# Précision par catégorie
accuracy_fiction = df_eval[df_eval["vraie_categorie"] == "Fiction"]["correct"].mean()
accuracy_nonfiction = df_eval[df_eval["vraie_categorie"] == "Nonfiction"]["correct"].mean()

print(f"Total de prédictions: {total}")
print(f"Prédictions correctes: {corrects}")
print(f"Prédictions incorrectes: {total - corrects}")
print(f"\nAccuracy global: {accuracy:.2%}")
print(f"Accuracy Fiction: {accuracy_fiction:.2%}")
print(f"Accuracy Nonfiction: {accuracy_nonfiction:.2%}")
print(f"Score de confiance moyen: {df_eval['score'].mean():.2%}")


RÉSULTATS D'ÉVALUATION DU MODÈLE
Total de prédictions : 600
Prédictions correctes : 467
Prédictions incorrectes : 133

Accuracy global : 77.83%
Accuracy Fiction : 68.00%
Accuracy Nonfiction : 87.67%
Score de confiance moyen : 71.89%


In [21]:
# Trouver les livres sans catégorie
livres_manquants = df[df["cat_simple"].isnull()].copy().reset_index(drop=True)

print(f"Nombre de livres sans catégorie: {len(livres_manquants)}")
print("\nAperçu:")
print(livres_manquants[["isbn13", "title", "categories"]].head(10))


Nombre de livres sans catégorie : 1454

Aperçu :
          isbn13                     title  \
0  9780002261982              Spider's Web   
1  9780006280897            The Four Loves   
2  9780006280934       The Problem of Pain   
3  9780006380832    Empires of the Monsoon   
4  9780006470229      The Gap Into Madness   
5  9780006472612        Master of the Game   
6  9780006483014  The Once and Future King   
7  9780006483892           Murder in LaMut   
8  9780006483908            Jimmy the Hand   
9  9780006486145          Well of Darkness   

                            categories  
0        Detective and mystery stories  
1                       Christian life  
2                       Christian life  
3                         Africa, East  
4  Hyland, Morn (Fictitious character)  
5                    Adventure stories  
6                   Arthurian romances  
7                    Adventure stories  
8                      Fantasy fiction  
9                                 

In [22]:
print(f"\nClassification de {len(livres_manquants)} livres manquants")

categories_predites_manquantes = []
isbn_manquants = []

# Classifier chaque livre sans catégorie
for idx, row in tqdm(livres_manquants.iterrows(), total=len(livres_manquants)):
    desc = row["description"]
    isbn = row["isbn13"]

    cat_pred, _ = classifier_description(desc, ["Fiction", "Nonfiction"])

    categories_predites_manquantes.append(cat_pred)
    isbn_manquants.append(isbn)


Classification de 1454 livres manquants...


100%|██████████| 1454/1454 [42:27<00:00,  1.75s/it] 


In [23]:
# Créer un DataFrame avec les prédictions
df_pred_manquantes = pd.DataFrame({
    "isbn13": isbn_manquants,
    "cat_predite": categories_predites_manquantes
})

print("Prédictions effectuées:")
print(df_pred_manquantes["cat_predite"].value_counts())

Prédictions effectuées:
cat_predite
Nonfiction    1010
Fiction        444
Name: count, dtype: int64


In [24]:
# Fusionner les prédictions avec le dataset original
df_final = df.merge(df_pred_manquantes, on="isbn13", how="left")

# Remplir les catégories manquantes avec les prédictions
df_final["cat_simple"] = df_final["cat_simple"].fillna(df_final["cat_predite"])

# Supprimer la colonne temporaire
df_final = df_final.drop(columns=["cat_predite"])

print("Fusion effectuée avec succès")
print(f"Livres avec catégorie maintenant: {df_final['cat_simple'].notna().sum()}")


Fusion effectuée avec succès
Livres avec catégorie maintenant : 5197


In [25]:
# Vérifier les valeurs manquantes restantes
manquantes_restantes = df_final["cat_simple"].isnull().sum()
print(f"Livres toujours sans catégorie: {manquantes_restantes}")

# Distribution finale
print("\nDistribution finale des catégories simplifiées:")
print(df_final["cat_simple"].value_counts())

Livres toujours sans catégorie : 0

Distribution finale des catégories simplifiées :
cat_simple
Fiction              2808
Nonfiction           1942
Fiction enfant        390
Nonfiction enfant      57
Name: count, dtype: int64


In [26]:
# Lister les genres spécialisés à rechercher
genres_cibles = [
    "romance", "science fiction", "scifi", "fantasy",
    "horror", "mystery", "thriller", "comedy", "crime", "historical"
]

# Filtrer les livres par genres (insensible à la casse)
livres_genres = df_final[
    df_final["categories"].str.lower().isin(genres_cibles)
].copy()

print(f"Livres trouvés dans les genres spécialisés: {len(livres_genres)}")
print("\nRépartition par genre:")
print(livres_genres["categories"].value_counts())

Livres trouvés dans les genres spécialisés : 16

Répartition par genre :
categories
Science fiction    8
Fantasy            6
Comedy             1
Horror             1
Name: count, dtype: int64


In [28]:
# Sauvegarder dans un fichier CSV
df_final.to_csv("livres_avec_categories.csv", index=False)

print("\nFichier sauvegardé")


Fichier sauvegardé


In [29]:
# Un récapitulatif de ce qui a été fait
print(f"Total de livres traités: {len(df_final)}")
print(f"Livres avec catégorie: {df_final['cat_simple'].notna().sum()}")
print(f"Livres sans catégorie: {df_final['cat_simple'].isna().sum()}")

print("\nRépartition finale:")
repartition = df_final["cat_simple"].value_counts()
for cat, count in repartition.items():
    pourcentage = (count / len(df_final)) * 100
    print(f"  {cat:20}: {count:6} livres ({pourcentage:5.1f}%)")

print("\nGenres spécialisés identifiés:")
for genre in genres_cibles:
    count = len(livres_genres[livres_genres["categories"].str.lower() == genre])
    if count > 0:
        print(f"  {genre:20}: {count:6} livres")

Total de livres traités : 5197
Livres avec catégorie : 5197
Livres sans catégorie : 0

Répartition finale :
  Fiction              :   2808 livres ( 54.0%)
  Nonfiction           :   1942 livres ( 37.4%)
  Fiction enfant       :    390 livres (  7.5%)
  Nonfiction enfant    :     57 livres (  1.1%)

Genres spécialisés identifiés :
  science fiction      :      8 livres
  fantasy              :      6 livres
  horror               :      1 livres
  comedy               :      1 livres
